### Requests Spider for "ГИВЦ Минобрнауки России"  

Сайт: https://miccedu.ru/

In [1]:
#2017
url_2017 = 'http://indicators.miccedu.ru/monitoring/_vpo/inst.php?id='
# 2016 архив
url_2016 = 'http://indicators.miccedu.ru/monitoring/2016/_vpo/inst.php?id='
# 2015 архив
url_2015 = 'http://indicators.miccedu.ru/monitoring/2015/inst.php?id='
# 2014 архив
url_2014 = 'https://miccedu.ru/monitoring/2014/materials/inst_'

In [2]:
import requests
import codecs
import pandas as pd
import re
from re import finditer
from lxml import html
from tqdm import tqdm

In [3]:
class Spider(object):
    
    """
    ################################################################################
    Парсер архивов с информацией с сайта https://miccedu.ru/ ГИВЦ Минобрнауки России
    ################################################################################
    """
    
    def __init__(self, year):       
        self.year = year         # год загрузки архивных данных
        
    def get_templates(self, year):
        # Словарь мета-данных - общих сведений образовательной организации
        
        if year != 2014:
            edu_dict = {
                        'Наименование образовательной организации': None,
                        'Регион':None,
                        'Ректор':None,
                        'Ведомственная принадлежность':None,
                        'web-сайт':None,
                        'Профиль организации':None,
                        'Результат оценки эффективности деятельности': None
                       }
        else:
             edu_dict = {
                        'Наименование образовательной организации': None,
                        'Регион': None,
                        'адрес': None,
                        'Ведомственная принадлежность': None,
                        'web-сайт': None,
                        'Учредитель(и)': None,
                        'Профиль организации': None
                       }
            
        if year == 2016 or year == 2017:
            pull_1 = "<tr><td>(.*)</td><td class=\'n\'>(.*)</td>\r\n          <td>(.*)</td><td>(.*)</td>\r\n"
            pull_2 = "<tr><td>(.*)</td><td class=\'n\'>(.*)</td>\r\n          <td>(.*)</td>\r\n          <td>(.*)</td>\r\n"
            pull_3 = "<tr><td>(.*)</td>(.*)<td class=\'n\'>(.*)</td>(.*)<td style=\'vertical-align:bottom;\'>(.*)</td>(.*)<td style=\'vertical-align:bottom;\'>(.*)</td></tr>\r\n"  
            pull_3_add = "<tr><td rowspan=\'4\'>(.*)</td><td class=\'n\'>(.*)</td>(.*)<td style=\'vertical-align:bottom;\'>(.*)</td>(.*)<td style=\'vertical-align:bottom;\'>(.*)</td></tr>\r\n(.*)<tr>(.*)<td class=\'n\' style=\'padding-left:20px;\'>(.*):<br />(.*)</td>(.*)<td style=\'vertical-align:bottom;\'>(.*)</td>(.*)<td style=\'vertical-align:bottom;\'>(.*)</td></tr>\r\n(.*)<tr>(.*)<td class=\'n\' style=\'padding-left:20px;\'>(.*)</td>(.*)<td style=\'vertical-align:bottom;\'>(.*)</td>(.*)<td style=\'vertical-align:bottom;\'>(.*)</td></tr>\r\n(.*)<tr(.*)<td class=\'n\' style=\'padding-left:20px;\'>(.*)</td>(.*)<td style=\'vertical-align:bottom;\'>(.*)</td>(.*)<td style=\'vertical-align:bottom;\'>(.*)</td></tr>\r\n"
            add_params = None
            
        elif year == 2015:
            pull_1 = "<tr><td>(.*)</td><td class=\'n\'>(.*)</td>\r\n          <td>(.*)</td><td>(.*)</td>\r\n"
            pull_2 = "<tr><td>(.*)</td><td class=\'n\'>(.*)</td>\r\n          <td>(.*)</td>\r\n          <td>(.*)</td>\r\n"
            pull_3 = "<td>(.*)</td>\r\n(.*)<td class=\'n\'>(.*)</td>\r\n(.*)<td style=\'vertical-align:bottom;\'>(.*)</td>\r\n(.*)<td style=\'vertical-align:bottom;\'>(.*)</td></tr>\r\n"
            pull_3_add = "<td rowspan=\'4\'>(.*)</td>\r\n(.*)<td class=\'n\'>(.*)</td>\r\n(.*)<td style=\'vertical-align:bottom;\'>(.*)</td>\r\n(.*)<td style=\'vertical-align:bottom;\'>(.*)</td></tr>\r\n(.*)<tr>\r\n(.*)<td class=\'n\' style=\'padding-left:20px;\'>(.*)<br />(.*)</td>\r\n(.*)<td style=\'vertical-align:bottom;\'>(.*)</td>\r\n(.*)<td style=\'vertical-align:bottom;\'>(.*)</td></tr>\r\n(.*)<tr>\r\n(.*)<td class=\'n\' style=\'padding-left:20px;\'>(.*)</td>\r\n(.*)<td style=\'vertical-align:bottom;\'>(.*)</td>\r\n(.*)<td style=\'vertical-align:bottom;\'>(.*)</td></tr>\r\n(.*)<tr>\r\n(.*)<td class=\'n\' style=\'padding-left:20px;\'>(.*)</td>\r\n(.*)td style=\'vertical-align:bottom;\'>(.*)</td>\r\n(.*)<td style=\'vertical-align:bottom;\'>(.*)</td></tr>\r\n"
            add_params = None
            
        elif year == 2014:
            
            #2014
            pull_1 = "<p class=MsoNormal style=\'margin-bottom:0cm;margin-bottom:.0001pt;line-height:\r\n  normal\'>(.*)</p>\r\n"
            pull_2 = "text-align:right;line-height:normal\'(.*)</p>\r\n"
            
            pull_2_2014_a = "<span style=\'font-size:10.0pt\'>1</span></p>\r\n  </td>\r\n  <td valign=top style=\'border-top:none;border-left:none;border-bottom:solid windowtext 1.0pt;\r\n  border-right:solid windowtext 1.0pt;padding:0cm 5.4pt 0cm 5.4pt;height:5.85pt\'>\r\n  <p class=MsoNormal style=\'margin-bottom:0cm;margin-bottom:.0001pt;line-height:\r\n  normal\'><span style=\'font-size:10.0pt\'>Общая численность студентов, обучающихся\r\n  по программам бакалавриата, специалитета, магистратуры</span></p>\r\n  </td>\r\n  <td valign=bottom style=\'border-top:none;border-left:none;border-bottom:solid windowtext 1.0pt;\r\n  border-right:solid windowtext 1.0pt;padding:0cm 5.4pt 0cm 5.4pt;height:5.85pt\'>\r\n  <p class=MsoNormal align=center style=\'margin-bottom:0cm;margin-bottom:.0001pt;\r\n  text-align:center;line-height:normal\'><span style=\'font-size:10.0pt\'>чел.</span></p>\r\n  </td>\r\n  <td valign=bottom style=\'border-top:none;border-left:none;border-bottom:solid windowtext 1.0pt;\r\n  border-right:solid windowtext 1.0pt;padding:0cm 5.4pt 0cm 5.4pt;height:5.85pt\'>\r\n  <p class=MsoNormal align=right style=\'margin-bottom:0cm;margin-bottom:.0001pt;\r\n  text-align:right;line-height:normal\'><span lang=EN-US style=\'font-size:10.5pt\'>"
            pull_2_2014_b = "<span style=\'font-size:10.0pt\'>1.1</span></p>\r\n  </td>\r\n  <td valign=top style=\'border-top:none;border-left:none;border-bottom:solid windowtext 1.0pt;\r\n  border-right:solid windowtext 1.0pt;padding:0cm 5.4pt 0cm 5.4pt;height:6.25pt\'>\r\n  <p class=MsoNormal style=\'margin-bottom:0cm;margin-bottom:.0001pt;line-height:\r\n  normal\'><span style=\'font-size:10.0pt\'>в том числе:</span></p>\r\n  <p class=MsoNormal style=\'margin-bottom:0cm;margin-bottom:.0001pt;line-height:\r\n  normal\'><span style=\'font-size:10.0pt\'>\xa0\xa0\xa0 по очной форме обучения</span></p>\r\n  </td>\r\n  <td valign=bottom style=\'border-top:none;border-left:none;border-bottom:solid windowtext 1.0pt;\r\n  border-right:solid windowtext 1.0pt;padding:0cm 5.4pt 0cm 5.4pt;height:6.25pt\'>\r\n  <p class=MsoNormal align=center style=\'margin-bottom:0cm;margin-bottom:.0001pt;\r\n  text-align:center;line-height:normal\'><span style=\'font-size:10.0pt\'>чел.</span></p>\r\n  </td>\r\n  <td valign=bottom style=\'border-top:none;border-left:none;border-bottom:solid windowtext 1.0pt;\r\n  border-right:solid windowtext 1.0pt;padding:0cm 5.4pt 0cm 5.4pt;height:6.25pt\'>\r\n  <p class=MsoNormal align=right style=\'margin-bottom:0cm;margin-bottom:.0001pt;\r\n  text-align:right;line-height:normal\'><span lang=EN-US style=\'font-size:10.5pt\'>"
            pull_2_2014_c = "по очно-заочной"
            pull_2_2014_d = "<span style=\'font-size:10.0pt\'>1.3</span></p>\r\n  </td>\r\n  <td valign=top style=\'border-top:none;border-left:none;border-bottom:solid windowtext 1.0pt;\r\n  border-right:solid windowtext 1.0pt;padding:0cm 5.4pt 0cm 5.4pt\'>\r\n  <p class=MsoNormal style=\'margin-bottom:0cm;margin-bottom:.0001pt;line-height:\r\n  normal\'><span style=\'font-size:10.0pt\'>\xa0\xa0\xa0 по заочной форме обучения</span></p>\r\n  </td>\r\n  <td valign=bottom style=\'border-top:none;border-left:none;border-bottom:solid windowtext 1.0pt;\r\n  border-right:solid windowtext 1.0pt;padding:0cm 5.4pt 0cm 5.4pt\'>\r\n  <p class=MsoNormal align=center style=\'margin-bottom:0cm;margin-bottom:.0001pt;\r\n  text-align:center;line-height:normal\'><span style=\'font-size:10.0pt\'>чел.</span></p>\r\n  </td>\r\n  <td valign=bottom style=\'border-top:none;border-left:none;border-bottom:solid windowtext 1.0pt;\r\n  border-right:solid windowtext 1.0pt;padding:0cm 5.4pt 0cm 5.4pt\'>\r\n  <p class=MsoNormal align=right style=\'margin-bottom:0cm;margin-bottom:.0001pt;\r\n  text-align:right;line-height:normal\'><span lang=EN-US style=\'font-size:10.5pt\'>"
            pull_2_2014_e = "<span style=\'font-size:10.0pt\'>1.4</span></p>\r\n  </td>\r\n  <td valign=top style=\'border-top:none;border-left:none;border-bottom:solid windowtext 1.0pt;\r\n  border-right:solid windowtext 1.0pt;padding:0cm 5.4pt 0cm 5.4pt\'>\r\n  <p class=MsoNormal style=\'margin-bottom:0cm;margin-bottom:.0001pt;line-height:\r\n  normal\'><span style=\'font-size:10.0pt\'>\xa0\xa0\xa0 экстернат</span></p>\r\n  </td>\r\n  <td valign=bottom style=\'border-top:none;border-left:none;border-bottom:solid windowtext 1.0pt;\r\n  border-right:solid windowtext 1.0pt;padding:0cm 5.4pt 0cm 5.4pt\'>\r\n  <p class=MsoNormal align=center style=\'margin-bottom:0cm;margin-bottom:.0001pt;\r\n  text-align:center;line-height:normal\'><span style=\'font-size:10.0pt\'>чел.</span></p>\r\n  </td>\r\n  <td valign=bottom style=\'border-top:none;border-left:none;border-bottom:solid windowtext 1.0pt;\r\n  border-right:solid windowtext 1.0pt;padding:0cm 5.4pt 0cm 5.4pt\'>\r\n  <p class=MsoNormal align=right style=\'margin-bottom:0cm;margin-bottom:.0001pt;\r\n  text-align:right;line-height:normal\'><span lang=EN-US style=\'font-size:10.5pt\'>"
            pull_2_2014_f = "<span lang=EN-US style=\'font-size:10.0pt\'>2</span></p>\r\n  </td>\r\n  <td valign=top style=\'border-top:none;border-left:none;border-bottom:solid windowtext 1.0pt;\r\n  border-right:solid windowtext 1.0pt;padding:0cm 5.4pt 0cm 5.4pt\'>\r\n  <p class=MsoNormal style=\'margin-bottom:0cm;margin-bottom:.0001pt;line-height:\r\n  normal\'><span style=\'font-size:10.0pt\'>Средний балл ЕГЭ студентов, принятых\r\n  на обучение по программам бакалавриата и специалитета, по всем формам\r\n  обучения</span></p>\r\n  </td>\r\n  <td valign=bottom style=\'border-top:none;border-left:none;border-bottom:solid windowtext 1.0pt;\r\n  border-right:solid windowtext 1.0pt;padding:0cm 5.4pt 0cm 5.4pt\'>\r\n  <p class=MsoNormal align=center style=\'margin-bottom:0cm;margin-bottom:.0001pt;\r\n  text-align:center;line-height:normal\'><span style=\'font-size:10.0pt\'>балл</span></p>\r\n  </td>\r\n  <td valign=bottom style=\'border-top:none;border-left:none;border-bottom:solid windowtext 1.0pt;\r\n  border-right:solid windowtext 1.0pt;padding:0cm 5.4pt 0cm 5.4pt\'>\r\n  <p class=MsoNormal align=right style=\'margin-bottom:0cm;margin-bottom:.0001pt;\r\n  text-align:right;line-height:normal\'><span lang=EN-US style=\'font-size:10.5pt\'>"
            pull_2_2014_g = "<span style=\'font-size:10.0pt\'>3</span></p>\r\n  </td>\r\n  <td valign=top style=\'border-top:none;border-left:none;border-bottom:solid windowtext 1.0pt;\r\n  border-right:solid windowtext 1.0pt;padding:0cm 5.4pt 0cm 5.4pt\'>\r\n  <p class=MsoNormal style=\'margin-bottom:0cm;margin-bottom:.0001pt;line-height:\r\n  normal\'><span style=\'font-size:10.0pt\'>Число предприятий, с которыми\r\n  заключены договоры на подготовку специалистов</span></p>\r\n  </td>\r\n  <td valign=bottom style=\'border-top:none;border-left:none;border-bottom:solid windowtext 1.0pt;\r\n  border-right:solid windowtext 1.0pt;padding:0cm 5.4pt 0cm 5.4pt\'>\r\n  <p class=MsoNormal align=center style=\'margin-bottom:0cm;margin-bottom:.0001pt;\r\n  text-align:center;line-height:normal\'><span style=\'font-size:10.0pt\'>ед.</span></p>\r\n  </td>\r\n  <td valign=bottom style=\'border-top:none;border-left:none;border-bottom:solid windowtext 1.0pt;\r\n  border-right:solid windowtext 1.0pt;padding:0cm 5.4pt 0cm 5.4pt\'>\r\n  <p class=MsoNormal align=right style=\'margin-bottom:0cm;margin-bottom:.0001pt;\r\n  text-align:right;line-height:normal\'><span style=\'font-size:10.5pt\'>"
            pull_2_2014_h = "<span style=\'font-size:10.0pt\'>4</span></p>\r\n  </td>\r\n  <td valign=top style=\'border-top:none;border-left:none;border-bottom:solid windowtext 1.0pt;\r\n  border-right:solid windowtext 1.0pt;padding:0cm 5.4pt 0cm 5.4pt\'>\r\n  <p class=MsoNormal style=\'margin-bottom:0cm;margin-bottom:.0001pt;line-height:\r\n  normal\'><span style=\'font-size:10.0pt\'>Число предприятий, являющихся базами\r\n  практики, с которыми оформлены договорные отношения</span></p>\r\n  </td>\r\n  <td valign=bottom style=\'border-top:none;border-left:none;border-bottom:solid windowtext 1.0pt;\r\n  border-right:solid windowtext 1.0pt;padding:0cm 5.4pt 0cm 5.4pt\'>\r\n  <p class=MsoNormal align=center style=\'margin-bottom:0cm;margin-bottom:.0001pt;\r\n  text-align:center;line-height:normal\'><span style=\'font-size:10.0pt\'>ед.</span></p>\r\n  </td>\r\n  <td valign=bottom style=\'border-top:none;border-left:none;border-bottom:solid windowtext 1.0pt;\r\n  border-right:solid windowtext 1.0pt;padding:0cm 5.4pt 0cm 5.4pt\'>\r\n  <p class=MsoNormal align=right style=\'margin-bottom:0cm;margin-bottom:.0001pt;\r\n  text-align:right;line-height:normal\'><span style=\'font-size:10.5pt\'>"
            pull_2_2014_i = "Общая численность работников\r\n  образовательного учреждения"
            pull_2_2014_j = "Общая численность ППС"
            pull_2_2014_k = "Общая площадь зданий"
            pull_2_2014_l = "<span lang=EN-US style=\'font-size:10.0pt\'>8</span></p>\r\n  </td>\r\n  <td valign=top style=\'border-top:none;border-left:none;border-bottom:solid windowtext 1.0pt;\r\n  border-right:solid windowtext 1.0pt;padding:0cm 5.4pt 0cm 5.4pt\'>\r\n  <p class=MsoNormal style=\'margin-bottom:0cm;margin-bottom:.0001pt;line-height:\r\n  normal\'><span style=\'font-size:10.0pt\'>Площадь учебно-лабораторных зданий</span></p>\r\n  </td>\r\n  <td valign=bottom style=\'border-top:none;border-left:none;border-bottom:solid windowtext 1.0pt;\r\n  border-right:solid windowtext 1.0pt;padding:0cm 5.4pt 0cm 5.4pt\'>\r\n  <p class=MsoNormal align=center style=\'margin-bottom:0cm;margin-bottom:.0001pt;\r\n  text-align:center;line-height:normal\'><span style=\'font-size:10.0pt\'>кв</span><span\r\n  lang=EN-US style=\'font-size:10.0pt\'>. </span><span style=\'font-size:10.0pt\'>м</span></p>\r\n  </td>\r\n  <td valign=bottom style=\'border-top:none;border-left:none;border-bottom:solid windowtext 1.0pt;\r\n  border-right:solid windowtext 1.0pt;padding:0cm 5.4pt 0cm 5.4pt\'>\r\n  <p class=MsoNormal align=right style=\'margin-bottom:0cm;margin-bottom:.0001pt;\r\n  text-align:right;line-height:normal\'><span lang=EN-US style=\'font-size:10.5pt\'>"
            pull_2_2014_m = "исследовательских подразделений"
            pull_2_2014_n = "<span lang=EN-US style=\'font-size:10.0pt\'>10</span></p>\r\n  </td>\r\n  <td valign=top style=\'border-top:none;border-left:none;border-bottom:solid windowtext 1.0pt;\r\n  border-right:solid windowtext 1.0pt;padding:0cm 5.4pt 0cm 5.4pt\'>\r\n  <p class=MsoNormal style=\'margin-bottom:0cm;margin-bottom:.0001pt;line-height:\r\n  normal\'><span style=\'font-size:10.0pt\'>Площадь общежитий</span></p>\r\n  </td>\r\n  <td valign=bottom style=\'border-top:none;border-left:none;border-bottom:solid windowtext 1.0pt;\r\n  border-right:solid windowtext 1.0pt;padding:0cm 5.4pt 0cm 5.4pt\'>\r\n  <p class=MsoNormal align=center style=\'margin-bottom:0cm;margin-bottom:.0001pt;\r\n  text-align:center;line-height:normal\'><span style=\'font-size:10.0pt\'>кв</span><span\r\n  lang=EN-US style=\'font-size:10.0pt\'>. </span><span style=\'font-size:10.0pt\'>м</span></p>\r\n  </td>\r\n  <td valign=bottom style=\'border-top:none;border-left:none;border-bottom:solid windowtext 1.0pt;\r\n  border-right:solid windowtext 1.0pt;padding:0cm 5.4pt 0cm 5.4pt\'>\r\n  <p class=MsoNormal align=right style=\'margin-bottom:0cm;margin-bottom:.0001pt;\r\n  text-align:right;line-height:normal\'><span lang=EN-US style=\'font-size:10.5pt\'>"
            pull_2_2014_o = "<span lang=EN-US style=\'font-size:10.0pt\'>11</span></p>\r\n  </td>\r\n  <td valign=top style=\'border-top:none;border-left:none;border-bottom:solid windowtext 1.0pt;\r\n  border-right:solid windowtext 1.0pt;padding:0cm 5.4pt 0cm 5.4pt\'>\r\n  <p class=MsoNormal style=\'margin-bottom:0cm;margin-bottom:.0001pt;line-height:\r\n  normal\'><span style=\'font-size:10.0pt\'>Площадь крытых спортивных сооружений</span></p>\r\n  </td>\r\n  <td valign=bottom style=\'border-top:none;border-left:none;border-bottom:solid windowtext 1.0pt;\r\n  border-right:solid windowtext 1.0pt;padding:0cm 5.4pt 0cm 5.4pt\'>\r\n  <p class=MsoNormal align=center style=\'margin-bottom:0cm;margin-bottom:.0001pt;\r\n  text-align:center;line-height:normal\'><span style=\'font-size:10.0pt\'>кв</span><span\r\n  lang=EN-US style=\'font-size:10.0pt\'>. </span><span style=\'font-size:10.0pt\'>м</span></p>\r\n  </td>\r\n  <td valign=bottom style=\'border-top:none;border-left:none;border-bottom:solid windowtext 1.0pt;\r\n  border-right:solid windowtext 1.0pt;padding:0cm 5.4pt 0cm 5.4pt\'>\r\n  <p class=MsoNormal align=right style=\'margin-bottom:0cm;margin-bottom:.0001pt;\r\n  text-align:right;line-height:normal\'><span lang=EN-US style=\'font-size:10.5pt\'>"
            pull_2_2014_p = "<span style=\'font-size:10.0pt\'>12</span></p>\r\n  </td>\r\n  <td valign=top style=\'border-top:none;border-left:none;border-bottom:solid windowtext 1.0pt;\r\n  border-right:solid windowtext 1.0pt;padding:0cm 5.4pt 0cm 5.4pt\'>\r\n  <p class=MsoNormal style=\'margin-bottom:0cm;margin-bottom:.0001pt;line-height:\r\n  normal\'><span style=\'font-size:10.0pt\'>Доля студентов, не обеспеченных\r\n  собственным общежитием вуза, в числе студентов, нуждающихся в общежитии</span></p>\r\n  </td>\r\n  <td valign=bottom style=\'border-top:none;border-left:none;border-bottom:solid windowtext 1.0pt;\r\n  border-right:solid windowtext 1.0pt;padding:0cm 5.4pt 0cm 5.4pt\'>\r\n  <p class=MsoNormal align=center style=\'margin-bottom:0cm;margin-bottom:.0001pt;\r\n  text-align:center;line-height:normal\'><span style=\'font-size:10.0pt\'>%</span></p>\r\n  </td>\r\n  <td valign=bottom style=\'border-top:none;border-left:none;border-bottom:solid windowtext 1.0pt;\r\n  border-right:solid windowtext 1.0pt;padding:0cm 5.4pt 0cm 5.4pt\'>\r\n  <p class=MsoNormal align=right style=\'margin-bottom:0cm;margin-bottom:.0001pt;\r\n  text-align:right;line-height:normal\'><span style=\'font-size:10.5pt\'>"
            pull_3 = [pull_2_2014_a, pull_2_2014_b,
                         pull_2_2014_c, pull_2_2014_d,
                         pull_2_2014_e, pull_2_2014_f,
                         pull_2_2014_g, pull_2_2014_h,
                         pull_2_2014_i, pull_2_2014_j,
                         pull_2_2014_k, pull_2_2014_l,
                         pull_2_2014_m, pull_2_2014_n,
                         pull_2_2014_o, pull_2_2014_p]
            pull_3_add = ['Средний балл ЕГЭ студентов, принятых по результатам ЕГЭ на обучение по очной форме по программам бакалавриата и специалитета за счет средств соответствующих бюджетов бюджетной системы РФ',
                          'Средний балл ЕГЭ студентов, принятых по результатам ЕГЭ на обучение по очной форме по программам бакалавриата и специалитета с оплатой стоимости затрат на обучение физическими и юридическими лицами',
                          'Усредненный по реализуемым направлениям (специальностям) минимальный балл ЕГЭ студентов, принятых по результатам ЕГЭ на обучение по очной форме на программы бакалавриата и специалитета',
                          'Численность студентов, победителей и призеров заключительного этапа всероссийской олимпиады школьников, членов сборных команд РФ, учувствовавших в международных олимпиадах, принятых на очную форму обучения на первый курс по программам бакалавриата и специалитета без вступительных испытаний',
                          'Численность студентов, победителей и призеров олимпиад школьников, принятых на очную форму обучения на первый курс по программам бакалавриата и специалитета по специальностям и (или) направлениям подготовки, соответствующим профилю олимпиады школьников, без вступительных испытаний',
                          'Численность студентов, принятых на условиях целевого приема на первый курс на очную форму обучения по программам бакалавриата и специалитета',
                          'Удельный вес численности студентов, принятых на условиях целевого приема на первый курс на очную форму обучения по программам бакалавриата и специалитета в общей численности студентов',
                          'Удельный вес численности студентов (приведенного контингента), обучающихся по программам магистратуры, в общей численности приведенного контингента студентов',
                          'Удельный вес численности студентов, имеющих диплом бакалавра, специалиста или магистра других организаций, принятых на первый курс на обучение по программам магистратуры образовательной организации, в общей численности студентов, принятых на первый курс по программам магистратуры на очную форму обучения',
                          'Численность аспирантов вуза в расчете на 100 студентов (приведенного контингента)',
                          'Удельный вес численности слушателей из сторонних организаций в общей численности слушателей, прошедших обучение в вузе по программам повышения квалификации и переподготовки',
                          'Количество цитирований статей в индексируемой системе цитирования Web of Science в расчете на 100 НПР',
                          'Количество цитирований статей в индексируемой системе цитирования Scopus в расчете на 100 НПР',
                          'Количество цитирований статей в Российском индексе научного цитирования (далее – РИНЦ) в расчете на 100 НПР',
                          'Количество статей в Web of Science, в расчете на 100 НПР',
                          'Количество статей в Scopus, в расчете на 100 НПР',
                          'Количество статей в РИНЦ, в расчете на 100 НПР',
                          'Общий объем научно-исследовательских и опытно-конструкторских работ (далее - НИОКР)',
                          'Удельный вес доходов от НИОКР в общих доходах образовательной организации',
                          'Удельный вес НИОКР, выполненных собственными силами (без привлечения соисполнителей), в общих доходах образовательной организации от НИОКР',
                          'Доходы от НИОКР (за исключением средств бюджетов бюджетной системы Российской Федерации, государственных фондов поддержки науки) в расчете на одного НПР',
                          'Количество лицензионных соглашений',
                          'Удельный вес средств, полученных вузом от управления объектами интеллектуальной собственности, в общих доходах вуза',
                          'Удельный вес численности НПР без ученой степени - до 30 лет, кандидатов наук - до 35 лет, докторов наук- до 40 лет, в общей численности НПР',
                          'Удельный вес научно-педагогических работников, защитивших кандидатские и докторские диссертации за отчетный год в общей численности НПР',
                          'Количество научных журналов, в том числе электронных, издаваемых вузом',
                          'Количество полученных грантов за отчетный год в расчете на 100 НПР',
                          'Удельный вес численности иностранных студентов (кроме стран СНГ), обучающихся программам бакалавриата, специалитета, магистратуры, в общей численности студентов (приведенный контингент)',
                          'Удельный вес численности иностранных студентов из СНГ, обучающихся по программам бакалавриата, специалитета, магистратуры, в общей численности студентов (приведенный контингент)',
                          'Удельный вес численности иностранных студентов, завершивших освоение образовательных программ бакалавриата, специалитета, магистратуры, в общей численности студентов (приведенный контингент)',
                          'Удельный вес численности иностранных студентов (кроме стран СНГ), завершивших освоение образовательных программ бакалавриата, программ специалитета, программ магистратуры, в общей численности студентов (приведенный контингент)',
                          'Удельный вес численности иностранных студентов из стран СНГ, завершивших освоение образовательных программ бакалавриата, программ специалитета, программ магистратуры, в общей численности студентов (приведенный контингент)',
                          'Удельный вес численности студентов вуза, обучающихся по очной форме обучения по образовательным программам бакалавриата, программам специалитета, программам магистратуры, прошедших обучение за рубежом не менее семестра (триместра), в общей численности студентов, обучающихся по очной форме обучения',
                          'Численность студентов иностранных образовательных организаций, прошедших обучение в вузе по образовательным по очной форме обучения по образовательным программам бакалавриата, программам специалитета, программам магистратуры, не менее семестра (триместра) в расчете на 100 студентов, обучающихся по очной форме обучения',
                          'Удельный вес численности иностранных граждан из числа НПР в общей численности НПР',
                          'Удельный вес численности иностранных граждан (кроме стран СНГ) из числа аспирантов вуза в общей численности аспирантов',
                          'Удельный вес численности иностранных граждан из стран СНГ из числа аспирантов вуза в общей численности аспирантов',
                          'Объем средств, полученных вузом на выполнение НИОКР от иностранных граждан и иностранных юридических лиц',
                          'Объем средств от образовательной деятельности, полученных вузом от иностранных граждан и иностранных юридических лиц',
                          'Доходы вуза из средств от приносящей доход деятельности в расчете на одного НПР',
                          'Отношение среднего заработка НПР в вузе (из всех источников) к средней заработной плате по экономике региона',
                          'Доходы вуза из всех источников в расчете на численность студентов (приведенный контингент)',
                          'Общая площадь учебно- лабораторных помещений в расчете на одного студента (приведенного контингента), в том числе:',
                          'имеющихся у вуза на праве собственности',
                          'закрепленных за вузом на праве оперативного управления',
                          'предоставленных вузу в безвозмездное пользование',
                          'предоставленных вузу в аренду',
                          'Количество персональных компьютеров в расчете на одного студента (приведенного контингента)',
                          'Удельный вес стоимости машин и оборудования (не старше 5 лет) вуза в общей стоимости машин и оборудования',
                          'Количество экземпляров печатных учебных изданий (включая учебники и учебные пособия) из общего количества единиц хранения библиотечного фонда, состоящих на учете, в расчете на одного студента (приведенного контингента)',
                          'Удельный вес выпускников отчетного года очной формы обучения, обратившихся за содействием в поиске подходящей работы в органы содействия в трудоустройстве',
                          'Удельный вес выпускников отчетного года очной формы обучения, обратившихся за содействием в поиске подходящей работы в органы содействия в трудоустройстве и признанных безработными',
                          'Удельный вес нетрудоустроенных в течение года выпускников отчетного года очной формы обучения, из числа обратившихся за содействием в трудоустройстве',
                          'Доля НПР, имеющих ученую степень кандидата наук, в общей численности НПР',
                          'Удельный вес НПР имеющих ученую степень доктора наук, в общей численности НПР',
                          'Удельный вес НПР, имеющих ученую степень кандидата и доктора наук, в общей численности НПР вуза (без совместителей и работающих по договорам гражданско-правового характера)',
                          'Число НПР, имеющих ученую степень кандидата и доктора наук, в расчете на 100 студентов',
                          'Доля штатных работников ППС в общей численности ППС']
        
            add_params =[
                   'Общая численность студентов, обучающихся по программам бакалавриата, специалитета, магистратуры чел.',
                   'в том числе по очной форме обучения чел.',
                   'в том числе   по очно-заочной (вечерней) форме обучения чел.',
                   'в том числе  по заочной форме обучения чел.',
                   'в том числе экстернат',
                   'Средний балл ЕГЭ студентов, принятых на обучение по программам бакалавриата и специалитета, по всем формам обучения',
                   'Число предприятий, с которыми заключены договоры на подготовку специалистов',
                   'Число предприятий, являющихся базами практики, с которыми оформлены договорные отношения',
                   'Общая численность работников образовательного учреждения (без внешних совместителей и работающих по договорам ГПХ)',
                   'Общая численность ППС (без внешних совместителей и работающих по договорам ГПХ)',
                   'Общая площадь зданий (помещений)' ,
                   'Площадь учебно-лабораторных зданий' ,
                   'Площадь, предназначенная для научно-исследовательских подразделений',
                   'Площадь общежитий' ,
                   'Площадь крытых спортивных сооружений' ,
                   'Доля студентов, не обеспеченных собственным общежитием вуза, в числе студентов, нуждающихся в общежитии'
                        ]
        else:
            print('Only 2014-2017')
            raise SystemExit()
        return edu_dict, pull_1, pull_2, pull_3, pull_3_add, add_params
    
    def clearstr(self, s):
        self.s = s
        ls = [x for x in list(":'></span></p></td>\r\n ")]
        for i in ls:
            s = s.replace(i,'')
        return s
                  
   
    def get_pages(self, url, ids_start, ids_stop):  
        print('Getting avaliable pages from the determined range')
        self.url = url
        self.ids_start = ids_start
        self.ids_stop = ids_stop
        headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36'}
        
        if self.year == 2014:
            add ='.htm'
        else:
            add=''
        
        page_ids = []
        for i in tqdm(range(ids_start ,ids_stop+1)):
            page = requests.get(url + str(i) + add, headers = headers)
            if re.search(
                         'Наименование образовательной организации',
                         codecs.decode(page.content, encoding='cp1251')
                        ):
                page_ids.append(i)
            else:
                pass
        print('Pages found: ',len(page_ids))
        return page_ids
    
    def load_params(self, url):
        
        page = requests.get(url)
        raw_data = codecs.decode(page.content, encoding='cp1251')
        edu_dict, pull_1, pull_2, pull_3, pull_3_add, add_params = self.get_templates(self.year)
        
        if self.year != 2014:
            
            start_pos = []
            for header in edu_dict:
                if header == 'Регион':
                    header = header + ',<br />адрес'
                else:
                    pass
                # try_catch для регорганизованных ун-в
                try:
                    m = [match.span()[1] for match in finditer(header + '</td>', raw_data)]
                    start_pos.append(m[0] if len(m)>0 else False)
                except:
                    pass

            position = []
            for pos in start_pos:
                # try_catch для регорганизованных ун-в
                try:
                    m = [match.span()[1] for match in finditer('</td>', raw_data[pos:])]
                    position.append(m[1] if len(m)>0 else False)
                except:
                    pass

            meta_info = []
            for i, x in enumerate(start_pos):
                srez = raw_data[x:]
                emp = []
                for match in finditer('</td>', srez):
                    emp.append(match.span()[1])

                if i == 4:
                    meta_info.append(str(re.sub(r"\s+", '', srez[:emp[0]])).replace('<td>','').replace('</td>',''))

                else:
                    meta_info.append(str(re.sub(r"\s+", " ", re.sub('([^А-Я а-я0-9])([^А-Я а-я0-9])', '', srez[:emp[0]])).replace(">","")))

            for header, info in zip(edu_dict, meta_info):
                edu_dict[header] = info

            df = pd.DataFrame([edu_dict], columns = edu_dict.keys())
            
            if self.year ==2017:
                
                df['Результат оценки эффективности деятельности'] = None

            else:
                df['Результат оценки эффективности деятельности'] = df['Результат оценки эффективности деятельности'].apply(lambda x: x[22:])
            
            try:
                df['Регион'] = df['Регион'].apply(lambda x: x.split('на картеv')[0][:-42][7:].split('r n')[0]+', '+x.split('на картеv')[0][:-42][7:].split('r n')[1][31:])
            except:
                pass
            
            try:
                df['Профиль организации'] = df['Профиль организации'].apply(lambda x: x[12 if self.year == 2016 or self.year == 2017 else 11:])
            except:
                pass
            
            try:
                df['Ведомственная принадлежность'] = df['Ведомственная принадлежность'].apply(lambda x: x[1:])
            except:
                pass
            
            try:
                df['Наименование образовательной организации'] = df['Наименование образовательной организации'].apply(lambda x: x[1:])
            except:
                pass
            
            try:
                df['Ректор'] = df['Ректор'].apply(lambda x: x[1:])
            except:
                pass

            # Подгрузка 1 пула показателей
            #########################################################
            for match in finditer(pull_1, raw_data):
                string = raw_data[match.span(0)[0]: match.span(0)[1]]
                m = re.match(pull_1, string)
                df[m.group(1) + '_' + m.group(2) + ' (' + m.group(3) + ')'] = m.group(4) 
            #########################################################

            # Подгрузка 2 пула показателей
            #########################################################
            for i,match in enumerate(finditer(pull_2, raw_data)):
                
                if (self.year ==2015) and (i==0):
                    pass
                else:
                    string = raw_data[match.span(0)[0]: match.span(0)[1]]
                    m = re.match(pull_2, string)
                    df[m.group(1) + '_' + m.group(2) + ' ('+ m.group(3) + ')'] = m.group(4)  
                    #########################################################

            # Подгрузка 3 пула показателей
            #########################################################
            for match in finditer(pull_3, raw_data):
                string = raw_data[match.span(0)[0]: match.span(0)[1]]
                m = re.match(pull_3, string)
                df[m.group(1) + '_' + m.group(3)] = m.group(7)
            #########################################################
            
            # Подгрузка декомпозированного показателя
            #########################################################
            if self.year == 2016 or self.year ==2017:
                
                for match in finditer(pull_3_add, raw_data):
                    string = raw_data[match.span(0)[0]: match.span(0)[1]]
                    m = re.match(pull_3_add, string)  
                    df[m.group(1) + '_' + m.group(2) + ' ('+ m.group(12) + ')'] = m.group(6)
                    for a, b, c, d, e in zip((1,1,1),(9,9,9),(10,17,24),(12,19,26),(14,21,28)):
                        df[m.group(a) + '_' + m.group(b) + ' ' + m.group(c) + ' (' + m.group(d) + ')'] = m.group(e)
            elif self.year == 2015:
                #2015 год
                for i, match in enumerate(finditer(pull_3_add, raw_data)):
                    if i <5:
                        string = raw_data[match.span(0)[0]: match.span(0)[1]]
                        m = re.match(pull_3_add, string) 
                        
                        df[m.group(1) + '_' + m.group(3) + ' ('+ m.group(5) + ')'] = m.group(7)
                        for a, b, c, d in zip((10,10,10),(11,18,25),(13,13,13),(15,22,29)):
                            df[m.group(a) + '_' + m.group(b) + ' ' + ' (' + m.group(c) + ')'] = m.group(d)
                    else:
                        pass
            #########################################################
        
        elif self.year == 2014:
            #2014
            meta_info= []

            for match in finditer('Наименование образовательной организации', raw_data):
                meta_info.append(raw_data[ match.span()[1]+257:].split(' </span></b></p>')[0].replace("&quot;", '"').replace('\r\n',''))
            
            for i,match in enumerate(finditer(pull_1, raw_data)):
                if i<7:
                    if i==3:
                        pass
                    else:
                        string = raw_data[match.span(0)[0]: match.span(0)[1]]
                        m = re.match(pull_1, string)
                        meta_info.append(str(m.group(1).replace('<span lang=EN-US>','').replace('</span>','')))
           

            df = pd.DataFrame([meta_info], columns = edu_dict.keys())
            
            # Подгрузка 1 пула показателей
            #########################################################
            digits = []
            garb = ['</span>', '<span lang=EN-US>', """text-align:right;line-height:normal'>""",
                    """<span lang=EN-US style='font-size:10.5pt'>""", '</p>',
                    """<span style='font-size:10.5pt'>"""]

            for i, match in enumerate(finditer(pull_2, raw_data)):

                if i <58: 
                    string = raw_data[match.span(0)[0]: match.span(0)[1]]
                    m = re.match(pull_2, string)
                    #df[m.group(1) + '_' + m.group(3)] = m.group(7)
                    string = str(m.group(0))
                    for char in garb:
                        if char in string:
                            string = string.replace(char,'')
                    digits.append(self.clearstr(string))

            for key, value in zip(pull_3_add, digits):
                df[key] = value
            # Подгрузка 2 пула показателей
            #########################################################
            values = []
            for i,temp in enumerate(pull_3):  

                if i == 2:
                    try:
                        string = raw_data[re.search(temp, raw_data).end()+728:re.search(temp, raw_data).end()+753]
                        string = self.clearstr(string)
                    except:
                        string = 'nan'

                elif i == 8:
                    try:
                        string = raw_data[re.search(temp, raw_data).end()+761:re.search(temp, raw_data).end()+780]
                        string = self.clearstr(string)
                    except:
                        string = 'nan'
                    
                elif i == 9:
                    try:
                        string = raw_data[re.search(temp, raw_data).end()+761:re.search(temp, raw_data).end()+780]
                        string = self.clearstr(string)
                    except:
                        string = 'nan'
                        
                elif i ==10:
                    try:
                        string = raw_data[re.search(temp, raw_data).end()+803:re.search(temp, raw_data).end()+813]
                        string = self.clearstr(string)
                    except:
                        string = 'nan'
                        
                elif i ==12:
                    try:
                        string = raw_data[re.search(temp, raw_data).end()+791:re.search(temp, raw_data).end()+813]  
                        string = self.clearstr(string)
                    except:
                        string = 'nan'                  
                    
                else:
                    try:
                        string = raw_data[re.search(temp, raw_data).end():re.search(temp, raw_data).end()+10]
                        string = self.clearstr(string)
                    except:
                        string = 'nan' 

                values.append(string)
                
            for key, value in zip(add_params,values):
                df[key] = value

        
        return df

    def load_data(self, url_templ, ids_start, ids_stop):
        
        self.url_templ = url_templ
        self.ids_start = ids_start
        self.ids_stop = ids_stop
        
        if self.year == 2014:
            add ='.htm'
        else:
            add=''
            
        df_full = pd.DataFrame([])
        
        page_idxs = self.get_pages(url_templ,ids_start, ids_stop)
        
        print('Loading final data to dataset')
        for idx in tqdm(page_idxs):
            df = self.load_params(url_templ + str(idx) + add)
            
            df['site_id'] = idx           
            df_full = df_full.append(df, sort = False)
            
        df_full = df_full.reset_index(drop=True)
        return df_full

In [6]:
sp = Spider(year = 2015)
df_un = sp.load_data(url_templ = url_2015,
                     ids_start = 0,
                     ids_stop = 5
                    )

Getting avaliable pages from the determined range


100%|██████████| 6/6 [00:01<00:00,  3.74it/s]


Pages found:  5
Loading final data to dataset


100%|██████████| 5/5 [00:01<00:00,  2.71it/s]


In [7]:
df_un

,Наименование образовательной организации,Регион,Ректор,Ведомственная принадлежность,web-сайт,Профиль организации,Результат оценки эффективности деятельности,"1.1_Средний балл ЕГЭ студентов, принятых по результатам ЕГЭ на обучение по очной форме по программам бакалавриата и специалитета за счет средств соответствующих бюджетов бюджетной системы РФ (балл)","1.2_Средний балл ЕГЭ студентов университета, принятых по результатам ЕГЭ на обучение по очной форме по программам бакалавриата и специалитета за счет средств соответствующих бюджетов бюджетной системы Российской Федерации, за исключением лиц, поступивших с учетом особых прав и в рамках квоты целевого приема (балл)","1.3_Средний балл ЕГЭ студентов, принятых по результатам ЕГЭ на обучение по очной форме по программам бакалавриата и специалитета с оплатой стоимости затрат на обучение физическими и юридическими лицами (балл)",...,52_Доля доходов вуза из бюджета субъекта РФ и местного бюджета,53_Доля доходов вуза от образовательной деятельности в общих доходах вуза,54_Доля доходов вуза от научных исследований и разработок в общих доходах вуза,55_Доля внебюджетных средств в доходах от образовательной деятельности,56_Доля внебюджетных средств в доходах от научных исследований и разработок,"1_Общая численность студентов, обучающихся по программам бакалавриата, специалитета, магистратуры (чел.)",1_в том числе по очной форме обучения (чел.),1_в том числе по очно-заочной (вечерней) форме обучения (чел.),1_в том числе по заочной форме обучения (чел.),site_id
0,Федеральное государственное бюджетное образова...,"Республика Адыгея, Республика Адыгея, г. Майк...",Хунагов Рашид Думаличевич,Министерство образования и науки Российской Фе...,http://adygnet.ru,"образовательные организации, не имеющие специф...",выполнил 4 или более показателей,"58,58","58,58","51,18",...,"0,23","55,60","3,00","26,00","59,41",5 065,3 236,270,1 559,1
1,Федеральное государственное бюджетное образова...,"Алтайский край, Алтайский край, г. Барнаул, у...",Лазаренко Ирина Рудольфовна,Министерство образования и науки Российской Фе...,http://uni-altai.ru,"образовательные организации, не имеющие специф...",выполнил 4 или более показателей,"62,58","62,58","54,70",...,"0,20","63,17","5,26","18,01","93,17",6 573,3 103,171,3 299,2
2,федеральное государственное бюджетное образова...,"Алтайский край, Алтайский край, г.Барнаул, пр...",Ситников Александр Андреевич,Министерство образования и науки Российской Фе...,http://www.altstu.ru,"образовательные организации, не имеющие специф...",выполнил 4 или более показателей,"60,49","60,45","56,09",...,"0,00","89,76","4,44","25,01","80,12",11 152,6 917,350,3 885,3
3,федеральное государственное бюджетное образова...,"Алтайский край, Алтайский край, г.Барнаул, пр...",Аничкин Евгений Сергеевич,Министерство образования и науки Российской Фе...,http://www.asu.ru/,"образовательные организации, не имеющие специф...",выполнил 4 или более показателей,"68,21","67,54","58,80",...,"0,00","73,84","12,19","38,15","24,72",10 600,7 566,183,2 851,4
4,Федеральное государственное бюджетное образова...,"Амурская область, , Амурская область, г. Благо...",Плутенко Андрей Долиевич,Министерство образования и науки Российской Фе...,http://www.amursu.ru,"образовательные организации, не имеющие специф...",выполнил 4 или более показателей,"57,50","57,50","53,55",...,"0,00","84,47","4,82","32,55","87,82",5 385,3 506,80,1 799,5


In [ ]:
# сохранить в excel
df_un.to_excel('univers_2015.xlsx',index = False)

In [ ]:
#TODO
#2014#по 13 номеру-вузу сделать отладку 